# Simple FreeSurface example

Romain Beucher Nov 2016
romainbeucher@unimelb.edu.au

In [ ]:
import underworld as uw
import glucifer
from underworld import function as fn
import numpy as np
import os
import glob
import time as t

uw.__version__

In [ ]:
minX  = -1.0
maxX  =  1.0
meshV =  0.0

resX = 240
resY = 120

elementType="Q1/dQ0"

mesh = uw.mesh.FeMesh_Cartesian( elementType = (elementType), 
                                 elementRes  = ( resX, resY), 
                                 minCoord    = ( minX, 0.), 
                                 maxCoord    = ( maxX, 1.),
                                 periodic    = [False, False]  )



## Define some mesh Variables

In [ ]:
velocityField    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=mesh.dim )
pressureField    = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

### Setup the material swarm


In [ ]:
swarm  = uw.swarm.Swarm( mesh=mesh, particleEscape=True)
swarmLayout = uw.swarm.layouts.GlobalSpaceFillerLayout( swarm=swarm, particlesPerCell=50 )
swarm.populate_using_layout( layout=swarmLayout )

# create pop control object
pop_control = uw.swarm.PopulationControl(swarm, aggressive=True, particlesPerCell=50)

### Create a particle advection system

In [ ]:
advector = uw.systems.SwarmAdvector( swarm=swarm, velocityField=velocityField, order=2 )

## Add swarm Variable

In [ ]:
materialVariable = swarm.add_variable( dataType="int", count=1 )

# Material Distribution

In [ ]:
crust = 1
mantleLithosphere = 2
mantle = 3
anomaly = 4

coord = fn.input()

conditions = [ ( coord[1] > 0.95, crust ), 
               ( coord[1] > 0.85, mantleLithosphere),
               ( True, mantle)] 

materialVariable.data[:] = fn.branching.conditional( conditions ).evaluate(swarm)

for index, coord in enumerate(swarm.particleCoordinates.data):
    if coord[0] > -0.1 and coord[0] < 0.1 :
        if coord[1] > 0.6 and coord[1] < 0.8 :
            materialVariable.data[index] = 4

## Assign Materials Property

In [ ]:
densityMap   = { crust : 1.3, mantleLithosphere : 1.3, mantle : 1.3, anomaly: 0.9 }
rhoFn = fn.branching.map( fn_key = materialVariable, mapping = densityMap )

fn_force = rhoFn * (0.0,-1.0)


viscosityMap = { crust : 300.0, mantleLithosphere : 300.0, mantle : 1.0, anomaly: 500.0 }
viscosityFn = fn.branching.map( fn_key = materialVariable, mapping = viscosityMap )

# Boundary Conditions

In [ ]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
base   = mesh.specialSets["MinJ_VertexSet"]
top    = mesh.specialSets["MaxJ_VertexSet"]

allWalls = iWalls + jWalls

velocityBCs = uw.conditions.DirichletCondition( variable        = velocityField, 
                                                indexSetsPerDof = (iWalls, base) )

In [ ]:
figdef = glucifer.Figure( figsize=(1200,400))
figdef.append(glucifer.objects.Points(swarm,materialVariable,pointSize=5.))
figdef.append(glucifer.objects.VectorArrows(mesh=mesh, fn=velocityField, scaling=30))
figdef.append(glucifer.objects.Mesh(mesh))
figdef.show()

# Set up Stokes

In [ ]:
stokes = uw.systems.Stokes(    velocityField = velocityField, 
                               pressureField = pressureField,
                               conditions    = velocityBCs,
                               fn_viscosity  = viscosityFn, 
                               fn_bodyforce  = fn_force)

solver = uw.systems.Solver( stokes )

In [ ]:
from scipy.interpolate import interp1d


def updateMesh():
    # Extract top surface
    x = mesh.data[list(top)][:,0]
    y = mesh.data[list(top)][:,1]

    # Extract velocities from top
    vx = velocityField.data[list(top)][:,0]
    vy = velocityField.data[list(top)][:,1]

    # Advect top surface
    x2 = x + vx * dt
    y2 = y + vy * dt

    # Spline top surface
    f = interp1d(x2, y2, kind='cubic')
    xnew = x2
    ynew = f(xnew)

    yb = 0.0
    dh = ynew - y
    rh1 = y - yb # Model Height
    rh2 = ynew - yb

    dilat = rh2 / rh1

    with mesh.deform_mesh():
        node = 0
        for row in range(resY+1):
            index = 0
            for col in range(resX+1):
                mesh.data[node][1] = yb + dilat[index]*(mesh.data[node][1] - yb)
                index += 1
                node += 1


In [ ]:
step = 0
nsteps = 10

while step < nsteps:
    solver.solve()
    dt = advector.get_max_dt()
    # advect swarm
    advector.integrate(dt)
    pop_control.repopulate()
    
    print step, dt
    
    t0 = t.time()
    updateMesh()
    print("Time {}: ", t.time() - t0)
    
    figdef.save_image("image"+str(step)+".png")
    step+=1